# Non-Personalized RS

In this step we will implement the two main non-personalized Recommender Systems (RS). They are:

- Most-Popular: the one that recommends the most popular items
- Best-Rated: the one that recommends the items who was rated with higher grades

For this reason, you must do:

- Read the train file extracted from the dataset 
- Create a sparse matrix for the ratings file
- Implement the Most-Popular and save the items recommended
- Implement the Best-Rated and save the items recommended

In [1]:
# import libs
import operator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix
from collections import OrderedDict

# useful command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

plt.rcParams.update({'font.size': 14})

## Reading train and test files

You can read this file as you prefer. I propose to read the files by the pandas' library and create the sparse matrix after it.

In [2]:
df_train = pd.read_csv('../Dataset/ML-1M/trainSet.txt', sep='::', names=['userId', 'itemId', 'rating', 'timestamp'])
df_test = pd.read_csv('../Dataset/ML-1M/testSet.txt', sep='::', names=['userId', 'itemId', 'rating', 'timestamp'])

df_train.head()
df_test.head()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,userId,itemId,rating,timestamp
0,1,1193,5.0,978300760.0
1,1,661,3.0,978302109.0
2,1,914,3.0,978301968.0
3,1,3408,4.0,978300275.0
4,1,1197,3.0,978302268.0


,userId,itemId,rating,timestamp
0,1,2355,5.0,978824291.0
1,1,595,5.0,978824268.0
2,1,2687,3.0,978824268.0
3,1,48,5.0,978824351.0
4,1,745,3.0,978824268.0


## Creating Sparse Matrix

I propose to use the csr_matrix from scipy.

In [3]:
# Select users, items and ratings logs (i.e., all information from each column)
users = df_train['userId']
items = df_train['itemId']
ratings = df_train['rating']

In [4]:
# Define the matrix dimensions based on the max index related to users and items
nb_users = max(users)
nb_items = max(items)

In [5]:
# Creating matrix of ratings
ratings_matrix = csr_matrix((ratings, (users, items)), shape=(nb_users+1, nb_items+1))

ratings_matrix.shape

(6041, 3953)

## An useful function

This function is used to save the recommendations in a file.

In [6]:
def dumpRecommendation(recommendation, users_targets, file_name):
    
    file_out = open(file_name, 'w')
    
    # for each user target
    for userId in users_targets:
        issuedItems = ""
        # for each item in the previous order
        for itemId in recommendation[userId]:
            issuedItems += str(itemId) + ":" + str(0.0) + ","
        # saving in file in correct format
        string_s = str(userId) + "\t" + "[" + issuedItems
        string_out = string_s[:-1] + ']'
        file_out.write(string_out + "\n")
    
    file_out.close()

## Most-Popular Recommendation

In [7]:
# Items popularity
items_popularity = {}

for i in range(ratings_matrix.shape[1]):
    items_popularity[i] = ratings_matrix[:,i].count_nonzero()

In [8]:
# Sorting the itemId descending by its popularity
popularity_order = OrderedDict(sorted(items_popularity.items(), key=operator.itemgetter(1), reverse=True))
most_popular = list(popularity_order.keys())

In [9]:
# Size of each recommendation
top_k = 10

In [10]:
# Setting the recommendations for each user
recommendation = {}

for u in range(ratings_matrix.shape[0]):
    # recommending just the most popular
#   recommendation[u] = most_popular[:top_k]
    recommendation[u] = []
    cont = 0
    # recommending the most popular that have never seen by users
    for i in most_popular:
        # recommending the top-k items 
        if (cont < top_k):
            if (ratings_matrix[u,i] == 0):
                recommendation[u].append(i)
                cont += 1
        else:
            break

In [11]:
# Save in a file
users_targets = df_test['userId'].unique()
dumpRecommendation(recommendation, users_targets, "../Recommendation/ML-1M/recList_MostPopular.txt")

In [12]:
recommendation[300]
recommendation[3000]
recommendation[6010]

[260, 1210, 480, 2571, 1580, 527, 1265, 1097, 2997, 2628]

[2028, 593, 608, 110, 2396, 527, 858, 1617, 2762, 318]

[2858, 480, 589, 2571, 593, 608, 2396, 1097, 1617, 2762]

In [13]:
recommendation[300]
recommendation[3000]
recommendation[6010]

[260, 1210, 480, 2571, 1580, 527, 1265, 1097, 2997, 2628]

[2028, 593, 608, 110, 2396, 527, 858, 1617, 2762, 318]

[2858, 480, 589, 2571, 593, 608, 2396, 1097, 1617, 2762]

## Best-Rated Recommendation

In [14]:
# Measure the mean of items' ratings
items_rating = {}

for i in range(ratings_matrix.shape[1]):
    items_rating[i] = np.mean(ratings_matrix[:,i])

In [15]:
# Sorting the itemId descending by its popularity
rating_order = OrderedDict(sorted(items_rating.items(), key=operator.itemgetter(1), reverse=True))
best_rated = list(rating_order.keys())

In [16]:
# Setting the recommendations for each user
recommendation = {}

for u in range(ratings_matrix.shape[0]):
    # recommending just the most popular
#   recommendation[u] = most_popular[:top_k]
    recommendation[u] = []
    cont = 0
    # recommending the most popular that have never seen by users
    for i in best_rated:
        # recommending the top-k items 
        if (cont < top_k):
            if (ratings_matrix[u,i] == 0):
                recommendation[u].append(i)
                cont += 1
        else:
            break

In [17]:
# Save in a file
users_targets = df_test['userId'].unique()
dumpRecommendation(recommendation, users_targets, "../Recommendation/ML-1M/recList_BestRated.txt")

In [18]:
recommendation[300]
recommendation[3000]
recommendation[6010]

[260, 1210, 2571, 527, 480, 1580, 2997, 1265, 296, 1097]

[2028, 593, 608, 527, 110, 858, 318, 2762, 2396, 1617]

[2858, 2571, 593, 589, 608, 318, 480, 2762, 2396, 1617]